_**Note:**_ If you don't have statements collected locally and you want it, execute this notebook.

# Collect Statements from LIAR Dataset

In [1]:
import pandas as pd
import pandas_profiling
import requests
import os
import json
import time
import datetime

In [2]:
df_train = pd.read_csv("../data/liar_dataset/train.tsv", sep='\t')
df_train.columns = ['statement_id', 'label', 'statement', 'subject', 'speaker', 'speakers_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']

In [3]:
df_valid = pd.read_csv("../data/liar_dataset/valid.tsv", sep='\t')
df_valid.columns = ['statement_id', 'label', 'statement', 'subject', 'speaker', 'speakers_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']

In [4]:
df = pd.concat([df_train, df_valid],ignore_index=True)
df.statement_id = df.statement_id.apply(lambda x: x[:-5])  # remove .json and get just ID

In [5]:
df.head(3)

,statement_id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,10540,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release


Execute pandas profiling tool on data we have (not including the statements' content):

In [6]:
profile = pandas_profiling.ProfileReport(df)
profile

Number of variables,14
Number of observations,11522
Total Missing (%),3.6%
Total size in memory,1.2 MiB
Average record size in memory,112.0 B
Numeric,2
Categorical,8
Boolean,0
Date,0
Text (Unique),1
Rejected,3


Let's see how many request we will need to make:

In [7]:
df.shape[0]

11522

---

# Collect rest of the data

In [8]:
# directory to which the JSON statements will be collected
directory_statements = "../data/liar_dataset/statements"

In [9]:
# form URL from statement ID
def get_URL(statement_id):
    return f"http://www.politifact.com/api/v/2/statement/{statement_id}/?format=json"

Following helper method `log_progress` will just provide the visual feedback information on how much items we already downloaded. This can be helpful since the collecting process takes some time.

In [10]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

Prepare methods for data collection:

In [11]:
def collect_id(session, sid):
    response = session.get(get_URL(sid))

    if response.content:
        with open(f"{directory_statements}/{sid}.json", "w") as f:
            json.dump(response.json(), f)
    else: 
        print(f"No content for statement id {sid}: {get_URL(sid)}")

def collect_data(directory, sids):
    sids = [sids] if type(sids) == str else sids
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with requests.Session() as s:
        for sid in log_progress(sids):
            collect_id(s, sid)

Start collecting:

In [12]:
start = time.time()
collect_data(directory=directory_statements, sids=df.statement_id)
end = time.time()

print(f"Total time needed: {str(datetime.timedelta(seconds=end - start))}")

No content for statement id 5355: http://www.politifact.com/api/v/2/statement/5355/?format=json
Total time needed: 0:59:11.554459


In [13]:
# # Parallelize requests
# def collect_id(session):
#     def inner_collect_id(sid):
#         response = session.get(get_URL(sid))
        
#         if response.content:
#             with open(f"{directory}/{sid}.json", "w") as f:
#                 json.dump(response.json(), f)
#         else: 
#             print(f"No content for statement id {sid}: {get_URL(sid)}")
    
#     return inner_collect_id
# def collect_data_parallel(directory, sids): 
#     with requests.Session() as s, Pool() as p:
#         print(p.map(collect_id(s), list(sids)))
        
# from multiprocessing import Pool
# start = time.time()
# collect_data_parallel(directory="../liar_dataset/statements", sids=df.statement_id)
# end = time.time()

# print(f"Total time needed: {end - start}")

---

# Check what we have
Let's see what have we collected loading the content of one file (e.g. statement id 2).

In [14]:
sid = 2
with open(f"{directory_statements}/{sid}.json", "r") as f:
    data = json.load(f)
data

{'art': [],
 'author': [{'email_address': 'bill.adair@duke.edu',
   'first_name': 'Bill',
   'id': 5,
   'last_name': 'Adair',
   'name_slug': 'bill-adair',
   'on_staff_page': None,
   'phone': None,
   'photo': 'https://s3.amazonaws.com/static.politifact.com/politifact/staffers/adair_80.jpg',
   'publication': {'id': 1,
    'publication_name': 'PolitiFact',
    'resource_uri': '/api/v/2/publication/1/'},
   'resource_uri': '/api/v/2/staffer/5/',
   'title': 'Former Editor',
   'twitter': None}],
 'canonical_url': '/truth-o-meter/statements/2007/oct/22/mitt-romney/defense-cuts-had-gop-support-too/',
 'edition': {'adkey': 'Editions',
  'advalue': 'National',
  'edition': 'National',
  'edition_slug': 'truth-o-meter',
  'has_afterthefact': False,
  'id': 1,
  'index_page': True,
  'meter_name': 'The Truth-O-Meter<sup>TM</sup>',
  'resource_uri': '/api/v/2/edition/1/'},
 'editor': [{'email_address': 'ahollyfield@tampabay.com',
   'first_name': 'Amy',
   'id': 36,
   'last_name': 'Hollyfi